|      |      | 
| ---- | ---- | 
| Journal:|Sustainability (ISSN 2071-1050)|
|Manuscript ID:| sustainability-1485080|
|Type:|Article|
|Title:|NEVs Supply Chain Coordination with Financial Constraint and Demand Uncertainty|
|manuscripts:|P.8 - figure 7. Data generation code|
|Code authors:|Yongjian Li$^*$ |
|Last modify:|2021-01-12|
|version:|Python 3.6.5, sympy >= 1.3|

In [1]:
from sympy import *
from sympy.stats import *
import numpy as np
import pandas as pd
# init_printing(use_unicode=True)
import sympy
sympy.__version__

'1.9'

In [2]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

In [3]:
# Initial parameter values
parameter = {p: 10, c: 4, s: 3, w: 6, theta: 0.8, b: 3, B: 100, r: 0.05,
             mu: 100, sigma: 100}

#### (see manuscript P(8) figure 7.)


\begin{align}
&F^{-1}(\alpha) \\
&\tilde{q}_{t}=\frac{(p-s) F^{-1}(\alpha)+B(1+r)}{c(1+r)-s} \\
&\tilde{q}_{s}=F^{-1}\left(\frac{p-c(1+r)}{p-s}\right)
\end{align}




- Suppose X is normally distributed with $\mu=100$,$\sigma=100$ 
- CDF,exp1: $F(x) = \alpha$

In [4]:
X = Normal(x, mu, sigma).subs(parameter)
exp1 = Eq(cdf(X)(x), alpha)
exp1

Eq(erf(sqrt(2)*(x - 100)/200)/2 + 1/2, alpha)

- inverse exp1: $F^{-1}(\alpha) = x ,\quad \{0\leqslant \alpha \leqslant 1\}$

In [5]:
i_cdf = solve(exp1, x)[0]
i_cdf

100*sqrt(2)*erfinv(2*alpha - 1) + 100

In [6]:
# Convert a SymPy expression into a function that allows for fast numeric evaluation.
f = lambdify(alpha, i_cdf, modules=['numpy', 'sympy'])

x_0 = [i/1000 for i in range(1, 1000)]

dt = {
    "x_0": x_0,
}

df = pd.DataFrame(dt)
df['f(x)'] = df['x_0'].map(f)
df1 = df.copy()
df1['f(x)'] = df1['f(x)'].map(int)
df1 = df1.drop_duplicates(subset=['f(x)']).reset_index(drop=True).copy()
df1

,x_0,f(x)
0,0.001,-209
1,0.002,-187
2,0.003,-174
3,0.004,-165
4,0.005,-157
...,...,...
422,0.995,357
423,0.996,365
424,0.997,374
425,0.998,387


In [7]:
df1.query("`f(x)`==0")

,x_0,f(x)
113,0.157,0


In [46]:
df1.iloc[::10].to_csv("inverse(CDF)",index=None,sep='\t')

$$\tilde{q}_{s}=F^{-1}\left(\frac{p-c(1+r)}{p-s}\right)$$

In [9]:
prob = (p-c*(1+r))/(p - s)
prob

(-c*(r + 1) + p)/(p - s)

In [10]:
prob =prob.subs(parameter).evalf(6)
prob

0.828571

In [12]:
eq = Eq(cdf(X)(a), prob)
qc = solve(eq, a)[0].subs(parameter)
qc = int(qc)
qc

194

In [ ]:
# 计算出CDF的反函数值后
expr1 = .subs(parameter)

f2 = lambdify(x, expr1.subs(parameter), modules=['numpy', 'sympy'])

qt_values = f2(f1)

$$\tilde{q}_{t}=\frac{(p-s) F^{-1}(\alpha)+B(1+r)}{c(1+r)-s}$$

In [20]:
q_t =((p-s) *x +B*(1+r))/(c*(1+r)-s)
q_t

(B*(r + 1) + x*(p - s))/(c*(r + 1) - s)

In [21]:
q_t1 = q_t.subs(parameter)

f2 = lambdify(x, q_t1, modules=['numpy', 'sympy'])

qd_t_values = f2(df['f(x)'])
qd_t_values = qd_t_values.to_numpy()
qd_t_values.shape

(999,)

In [32]:
dt = {
    "x_0": x_0,
    "f(x)": qd_t_values
}
df_q_t = pd.DataFrame(dt)
df_q_t0 = df_q_t.copy()
df_q_t['f(x)'] = df_q_t['f(x)'].map(int)
df_q_t.head(10)
df_q_t0

,x_0,f(x)
0,0.001,-1131.80217859789
1,0.002,-1008.09434780570
2,0.003,-932.039141509578
3,0.004,-876.207387942947
4,0.005,-831.733760403525
...,...,...
994,0.995,2173.40042707019
995,0.996,2217.87405460961
996,0.997,2273.70580817624
997,0.998,2349.76101447236


In [33]:
df_q_t = df_q_t.drop_duplicates(subset=['f(x)']).reset_index(drop=True)
df_q_t = df_q_t.iloc[::10].reset_index(drop=True)
df_q_t

,x_0,f(x)
0,0.001,-1131
1,0.011,-665
2,0.021,-515
3,0.031,-417
4,0.041,-343
...,...,...
95,0.951,1636
96,0.961,1698
97,0.971,1776
98,0.981,1881


In [41]:
value = solve(Eq(q_t1, qc), x)[0]
d1 = {
    "x_0": [cdf(X)(value).evalf()],
    "f(x)": [qc]
}
df_q_t = df_q_t.append(pd.DataFrame(d1))


value = solve(Eq(q_t1, 0), x)[0]
d1 = {
    "x_0": [cdf(X)(value).evalf()],
    "f(x)": [0]
}

df_q_t = df_q_t.append(pd.DataFrame(d1))
df_q_t = df_q_t.drop_duplicates(subset=['x_0'])
df_q_t = df_q_t.sort_values(by=['x_0']).reset_index(drop=True)
df_q_t

,x_0,f(x)
0,0.001,-1131
1,0.011,-665
2,0.021,-515
3,0.031,-417
4,0.041,-343
...,...,...
97,0.951,1636
98,0.961,1698
99,0.971,1776
100,0.981,1881


In [42]:
df_q_t_e = df_q_t.query("@qc >=`f(x)` >= 0").reset_index(drop=True)
df_q_t_e

,x_0,f(x)
0,0.125071935637150,0
1,0.131,16
2,0.141,43
3,0.151,68
4,0.161,93
5,0.171,116
6,0.181,139
7,0.191,160
8,0.201,181
9,0.206841776695705,194


In [47]:
df_q_t.to_csv("qt",index=None,sep='\t')
df_q_t_e.to_csv("qt_e",index=None,sep='\t')

- some help fun() note here.

In [5]:
## note here
## Given a value, calculate the cumulative probability
value = 0
cdf(X)(value).evalf()
## Given a value, calculate the cumulative probability
value = 189.380
P(X < value).evalf(subs=parameter)